# TrainingServingSkew
* Moniitors whether the alphas generated for inference are identical 
    to the alphas generated during training

In [1]:
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");

In [2]:
function read_training_alpha(alpha, split)
    uid = parse(Int, get_recommendee_username()) + 1
    df = read_alpha(alpha, "validation", true)
    mask = df.user .== uid
    df = RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
    df.user .= 1
    df
end;

In [3]:
function maximum_error(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    abs_errors = abs.(training.rating .- serving.rating[training.item])
    rel_errors = abs.(training.rating ./ serving.rating[training.item] .- 1)
    maximum(vcat(abs_errors, rel_errors))
end;

In [5]:
potential_alphas = readdir(recommendee_alpha_basepath())
alphas = [
    x for
    x in potential_alphas if ispath("$(recommendee_alpha_basepath())/$(x)/alpha.jld2")
]
ensemble_alphas = ["BPR", "ErrorExplicit", "ErrorImplicit", "Explicit", "NonlinearExplicit"]
inference_only_alphas = ["ItemCFRelated"]
standard_alphas = [x for x in alphas if x ∉ ensemble_alphas && x ∉ inference_only_alphas]

8-element Vector{String}:
 "ExplicitItemCF"
 "ExplicitUserItemBiases"
 "LinearExplicit"
 "LinearImplicit"
 "NeuralExplicitMatrixFactorization"
 "NeuralImplicitMatrixFactorization"
 "NeuralImplicitUserItemBiases"
 "SeenItemsCount"

In [8]:
for alpha in standard_alphas
    skew = maximum_error(alpha, "training")
    @info alpha, skew
end

[ Info: 20220630 02:01:27 ("ExplicitItemCF", 1.7881393f-6)
[ Info: 20220630 02:01:28 ("ExplicitUserItemBiases", 1.9073486f-6)
[ Info: 20220630 02:01:28 ("LinearExplicit", 0.59236336f0)
[ Info: 20220630 02:01:28 ("LinearImplicit", 0.4343292f0)
[ Info: 20220630 02:01:28 ("NeuralExplicitMatrixFactorization", 2.258233f0)
[ Info: 20220630 02:01:28 ("NeuralImplicitMatrixFactorization", 0.46423072f0)
[ Info: 20220630 02:01:28 ("NeuralImplicitUserItemBiases", 1.7881393f-7)
[ Info: 20220630 02:01:29 ("SeenItemsCount", 0.0f0)
